In [118]:
import pandas as pd 
import seaborn as sns

In [119]:
# load tips data from seaborn 
data = sns.load_dataset("tips")
print(data.shape)
data.head()

(244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [120]:
# splitting the data into two seperate df 
bill = data.groupby(["sex", "smoker"])["total_bill", "tip"].sum()
tip = bill.copy() 
# removing bill from tip and tip from bill 
del tip["total_bill"]
del bill["tip"]

In [121]:
bill.reset_index(inplace= True)
print(bill.shape)
bill.head()

(4, 3)


,sex,smoker,total_bill
0,Male,Yes,1337.07
1,Male,No,1919.75
2,Female,Yes,593.27
3,Female,No,977.68


In [122]:
tip.reset_index(inplace= True) 
print(tip.shape)
tip.head()

(4, 3)


,sex,smoker,tip
0,Male,Yes,183.07
1,Male,No,302.00
2,Female,Yes,96.74
3,Female,No,149.77


## Merging 

In [123]:
# Inner join 
inner_join = pd.merge(bill, tip, how="inner", on=["sex", "smoker"])
print(inner_join.shape)
inner_join.head()

(4, 4)


,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [124]:
# Left join
left_join = pd.merge(bill, tip.head(2), how= "left", on=["sex", "smoker"], indicator=True)
print(left_join.shape)
left_join.head()

(4, 5)


,sex,smoker,total_bill,tip,_merge
0,Male,Yes,1337.07,183.07,both
1,Male,No,1919.75,302.00,both
2,Female,Yes,593.27,NaN,left_only
3,Female,No,977.68,NaN,left_only


In [125]:
# Right join
right_join = pd.merge(bill, tip.head(2), how= "right", on=["sex", "smoker"], indicator= True)
print(right_join.shape)
right_join.head()

(2, 5)


,sex,smoker,total_bill,tip,_merge
0,Male,Yes,1337.07,183.07,both
1,Male,No,1919.75,302.00,both


In [126]:
# Outer join 
outer_join = pd.merge(bill.tail(2), tip.head(2), how= "outer", on=["sex", "smoker"], indicator=True)
print(outer_join.shape)
outer_join.head()

(4, 5)


,sex,smoker,total_bill,tip,_merge
0,Female,Yes,593.27,NaN,left_only
1,Female,No,977.68,NaN,left_only
2,Male,Yes,NaN,183.07,right_only
3,Male,No,NaN,302.00,right_only


In [127]:
# Setting indexes 
bill.set_index(keys= ["sex", "smoker"], inplace= True)
tip.set_index(keys= ["sex", "smoker"], inplace= True)

In [128]:
bill.head()

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [129]:
tip.head()

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [130]:
pd.merge?

Signature:
pd.merge(
    left,
    right,
    how: str = 'inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index: bool = False,
    right_index: bool = False,
    sort: bool = False,
    suffixes=('_x', '_y'),
    copy: bool = True,
    indicator: bool = False,
    validate=None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
    Object to merge with.
how : {'left', 'right', 'outer', 'inner'}, default 'inner'
    Type of merge to be performed.

    * left: use only keys from left frame, similar to a SQL left outer join;
      preserve key order.
    * right: use only keys from right frame, similar to a SQL right outer join;
      pres

In [133]:
# Merging on index
index_merge= pd.merge(bill, tip, how="inner", left_index=True, right_index=True)
index_merge.head()

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [134]:
# Merging partial index and column 
partial_merge= pd.merge(bill, tip, how="inner", left_index=True, right_on=["sex", "smoker"])
partial_merge.head()

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

## Concat

In [138]:
# Gluing dataframes row-wise 
row_glue= pd.concat([bill, tip], axis=0, sort=False)
row_glue.head(8)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [141]:
# Gluing dataframes column-wise
col_glue= pd.concat([bill, tip], axis=1, sort=False)
col_glue.head()

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [144]:
# Adding an indicator 
indicator_glue = pd.concat([bill, tip], axis=0, sort=False, keys=["df1", "df2"])
indicator_glue 

total_bill     tip
    sex    smoker                    
df1 Male   Yes        1337.07     NaN
           No         1919.75     NaN
    Female Yes         593.27     NaN
           No          977.68     NaN
df2 Male   Yes            NaN  183.07
           No             NaN  302.00
    Female Yes            NaN   96.74
           No             NaN  149.77